# Opmærksomhedsmekanismer og transformer-modeller

En stor ulempe ved rekurrente netværk er, at alle ord i en sekvens har samme indflydelse på resultatet. Dette fører til suboptimal ydeevne med standard LSTM encoder-decoder-modeller til sekvens-til-sekvens-opgaver, såsom navngiven entitetsgenkendelse og maskinoversættelse. I virkeligheden har specifikke ord i inputsekvensen ofte større indflydelse på sekventielle outputs end andre.

Overvej en sekvens-til-sekvens-model, såsom maskinoversættelse. Den implementeres af to rekurrente netværk, hvor det ene netværk (**encoder**) komprimerer inputsekvensen til en skjult tilstand, og det andet netværk (**decoder**) udfolder denne skjulte tilstand til det oversatte resultat. Problemet med denne tilgang er, at netværkets endelige tilstand har svært ved at huske begyndelsen af en sætning, hvilket resulterer i dårlig modelkvalitet for lange sætninger.

**Opmærksomhedsmekanismer** giver en metode til at vægte den kontekstuelle indflydelse af hver inputvektor på hver outputforudsigelse af RNN. Dette implementeres ved at skabe genveje mellem de mellemliggende tilstande i input-RNN og output-RNN. På denne måde, når vi genererer outputsymbolet $y_t$, tager vi højde for alle skjulte inputtilstande $h_i$ med forskellige vægtkoefficienter $\alpha_{t,i}$.

![Billede, der viser en encoder/decoder-model med et additivt opmærksomhedslag](../../../../../translated_images/encoder-decoder-attention.7a726296894fb567aa2898c94b17b3289087f6705c11907df8301df9e5eeb3de.da.png)
*Encoder-decoder-modellen med additiv opmærksomhedsmekanisme i [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), citeret fra [denne blogpost](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

Opmærksomhedsmatrixen $\{\alpha_{i,j}\}$ repræsenterer graden, som visse inputord spiller i genereringen af et givet ord i outputsekvensen. Nedenfor er et eksempel på en sådan matrix:

![Billede, der viser en prøvejustering fundet af RNNsearch-50, taget fra Bahdanau - arviz.org](../../../../../translated_images/bahdanau-fig3.09ba2d37f202a6af11de6c82d2d197830ba5f4528d9ea430eb65fd3a75065973.da.png)

*Figur taget fra [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Fig.3)*

Opmærksomhedsmekanismer er ansvarlige for meget af den nuværende eller næsten nuværende state-of-the-art inden for naturlig sprogbehandling. Tilføjelse af opmærksomhed øger dog kraftigt antallet af modelparametre, hvilket førte til skaleringsproblemer med RNN'er. En vigtig begrænsning ved skalering af RNN'er er, at modellernes rekurrente natur gør det udfordrende at batchbehandle og parallelisere træning. I en RNN skal hvert element i en sekvens behandles i sekventiel rækkefølge, hvilket betyder, at det ikke nemt kan paralleliseres.

Adoptionen af opmærksomhedsmekanismer kombineret med denne begrænsning førte til skabelsen af de nuværende state-of-the-art transformer-modeller, som vi kender og bruger i dag, fra BERT til OpenGPT3.

## Transformer-modeller

I stedet for at videresende konteksten af hver tidligere forudsigelse til det næste evalueringsskridt bruger **transformer-modeller** **positionelle kodninger** og **opmærksomhed** til at fange konteksten af et givet input inden for et angivet tekstvindue. Billedet nedenfor viser, hvordan positionelle kodninger med opmærksomhed kan fange konteksten inden for et givet vindue.

![Animeret GIF, der viser, hvordan evalueringer udføres i transformer-modeller.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

Da hver inputposition kortlægges uafhængigt til hver outputposition, kan transformere parallelisere bedre end RNN'er, hvilket muliggør meget større og mere udtryksfulde sprogmodeller. Hver opmærksomhedshoved kan bruges til at lære forskellige relationer mellem ord, hvilket forbedrer nedstrøms opgaver inden for naturlig sprogbehandling.

## Bygning af en simpel transformer-model

Keras indeholder ikke en indbygget transformer-lag, men vi kan bygge vores egen. Som før vil vi fokusere på tekstklassifikation af AG News-datasættet, men det er værd at nævne, at transformer-modeller viser de bedste resultater ved mere udfordrende NLP-opgaver.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

Nye lag i Keras bør underklasse `Layer`-klassen og implementere `call`-metoden. Lad os starte med **Positional Embedding**-laget. Vi vil bruge [noget kode fra den officielle Keras-dokumentation](https://keras.io/examples/nlp/text_classification_with_transformer/). Vi antager, at vi padder alle inputsekvenser til længden `maxlen`.


In [2]:
class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.maxlen = maxlen

    def call(self, x):
        maxlen = self.maxlen
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x+positions

Dette lag består af to `Embedding`-lag: ét til at indlejre tokens (på den måde, vi tidligere har diskuteret) og ét til at indlejre token-positioner. Token-positioner oprettes som en sekvens af naturlige tal fra 0 til `maxlen` ved hjælp af `tf.range` og sendes derefter gennem indlejringslaget. De to resulterende indlejringsvektorer lægges derefter sammen, hvilket giver en positionsindlejret repræsentation af input med formen `maxlen`$\times$`embed_dim`.

Nu skal vi implementere transformer-blokken. Den vil tage outputtet fra det tidligere definerede indlejringslag:


In [3]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, name='attn')
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

Transformer anvender `MultiHeadAttention` på den positionskodede input for at producere opmærksomhedsvektoren med dimensionen `maxlen`$\times$`embed_dim`, som derefter blandes med input og normaliseres ved hjælp af `LayerNormalization`.

> **Bemærk**: `LayerNormalization` ligner `BatchNormalization`, som blev diskuteret i *Computer Vision*-delen af denne læringssti, men den normaliserer output fra det forrige lag for hver træningsprøve uafhængigt for at bringe dem inden for området [-1..1].

Outputtet fra dette lag sendes derefter gennem et `Dense` netværk (i vores tilfælde - et to-lags perceptron), og resultatet lægges til det endelige output (som igen gennemgår normalisering).

Nu er vi klar til at definere den komplette transformer-model:


In [4]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = 256
vocab_size = 20000

model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_sequence_length=maxlen, input_shape=(1,)),
    TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim),
    TransformerBlock(embed_dim, num_heads, ff_dim),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(4, activation="softmax")
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 256)               0         
_________________________________________________________________
token_and_position_embedding (None, 256, 32)           648192    
_________________________________________________________________
transformer_block (Transform (None, 256, 32)           10656     
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                660       
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)               

In [5]:
print('Training tokenizer')
model.layers[0].adapt(ds_train.map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Training tokenizer
938/938 [==============================] - 45s 39ms/step - loss: 0.4978 - acc: 0.8068 - val_loss: 0.2808 - val_acc: 0.9124


## BERT Transformer Modeller

**BERT** (Bidirectional Encoder Representations from Transformers) er et meget stort multi-lags transformer-netværk med 12 lag for *BERT-base* og 24 for *BERT-large*. Modellen bliver først forudtrænet på en stor mængde tekstdata (Wikipedia + bøger) ved hjælp af usuperviseret træning (forudsige maskerede ord i en sætning). Under forudtræningen opnår modellen en betydelig forståelse af sproget, som derefter kan udnyttes med andre datasæt gennem finjustering. Denne proces kaldes **transfer learning**.

![billede fra http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362e39ee4381aab7cad06b5465a0b5f053a0f2aa05fbe14e746.da.png)

Der findes mange variationer af Transformer-arkitekturer, herunder BERT, DistilBERT, BigBird, OpenGPT3 og flere, som kan finjusteres.

Lad os se, hvordan vi kan bruge en forudtrænet BERT-model til at løse vores traditionelle sekvensklassifikationsproblem. Vi vil låne ideer og noget kode fra [den officielle dokumentation](https://www.tensorflow.org/text/tutorials/classify_text_with_bert).

For at indlæse forudtrænede modeller vil vi bruge **Tensorflow hub**. Først skal vi indlæse den BERT-specifikke vektorisering:


In [1]:
import tensorflow_text 
import tensorflow_hub as hub
vectorizer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

ModuleNotFoundError: No module named 'tensorflow_text'

In [7]:
vectorizer(['I love transformers'])

{'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[  101,  1045,  2293, 19081,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 

Det er vigtigt, at du bruger den samme vectorizer, som det oprindelige netværk blev trænet med. BERT-vectorizeren returnerer desuden tre komponenter:
* `input_word_ids`, som er en sekvens af token-numre for input-sætningen
* `input_mask`, der viser, hvilken del af sekvensen der indeholder faktisk input, og hvilken del der er padding. Dette ligner masken, der produceres af `Masking`-laget
* `input_type_ids` bruges til opgaver inden for sprogmodellering og gør det muligt at specificere to input-sætninger i én sekvens.

Derefter kan vi instantiere BERT-feature extractor:


In [8]:
bert = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1')

In [9]:
z = bert(vectorizer(['I love transformers']))
for i,x in z.items():
    print(f"{i} -> { len(x) if isinstance(x, list) else x.shape }")

pooled_output -> (1, 128)
encoder_outputs -> 4
sequence_output -> (1, 128, 128)
default -> (1, 128)


Så BERT-laget returnerer en række nyttige resultater:
* `pooled_output` er resultatet af at gennemsnitliggøre alle tokens i sekvensen. Du kan betragte det som en intelligent semantisk indlejring af hele netværket. Det svarer til outputtet fra `GlobalAveragePooling1D`-laget i vores tidligere model.
* `sequence_output` er outputtet fra det sidste transformer-lag (svarer til outputtet fra `TransformerBlock` i vores model ovenfor).
* `encoder_outputs` er outputtene fra alle transformer-lagene. Da vi har indlæst en 4-lags BERT-model (som du nok kan gætte ud fra navnet, der indeholder `4_H`), har den 4 tensorer. Den sidste er den samme som `sequence_output`.

Nu vil vi definere den end-to-end klassifikationsmodel. Vi vil bruge *funktionel modeldefinition*, hvor vi definerer modelinput og derefter angiver en række udtryk for at beregne dens output. Vi vil også gøre BERT-modelens vægte ikke-trænbare og kun træne den endelige klassifikator:


In [10]:
inp = keras.Input(shape=(),dtype=tf.string)
x = vectorizer(inp)
x = bert(x)
x = keras.layers.Dropout(0.1)(x['pooled_output'])
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
bert.trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

938/938 [==============================] - 528s 559ms/step - loss: 0.8056 - acc: 0.6983 - val_loss: 0.5953 - val_acc: 0.7888


Selvom der er få trænbare parametre, er processen ret langsom, fordi BERT-feature-ekstraktoren er beregningsmæssigt tung. Det ser ud til, at vi ikke kunne opnå rimelig nøjagtighed, enten på grund af manglende træning eller manglende modelparametre.

Lad os prøve at ophæve frysningen af BERT-vægtene og træne den også. Dette kræver en meget lille læringsrate og en mere omhyggelig træningsstrategi med **warmup**, ved brug af **AdamW**-optimering. Vi vil bruge `tf-models-official`-pakken til at oprette optimeringen:


In [12]:
from official.nlp import optimization 
bert.trainable=True
model.summary()
epochs = 3
opt = optimization.create_optimizer(
    init_lr=3e-5,
    num_train_steps=epochs*len(ds_train),
    num_warmup_steps=0.1*epochs*len(ds_train),
    optimizer_type='adamw')

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer=opt)
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

Som du kan se, går træningen ret langsomt - men du kan eksperimentere og træne modellen i nogle få epoker (5-10) og se, om du kan opnå det bedste resultat sammenlignet med de metoder, vi har brugt tidligere.

## Huggingface Transformers-bibliotek

En anden meget almindelig (og lidt enklere) måde at bruge Transformer-modeller på er [HuggingFace-pakken](https://github.com/huggingface/), som tilbyder simple byggesten til forskellige NLP-opgaver. Den er tilgængelig både for Tensorflow og PyTorch, en anden meget populær ramme for neurale netværk.

> **Note**: Hvis du ikke er interesseret i at se, hvordan Transformers-biblioteket fungerer - kan du springe til slutningen af denne notebook, da du ikke vil se noget væsentligt anderledes end det, vi har gjort ovenfor. Vi vil gentage de samme trin med at træne BERT-modellen ved hjælp af et andet bibliotek og en væsentligt større model. Processen indebærer derfor en ret lang træning, så du kan vælge blot at gennemgå koden.

Lad os se, hvordan vores problem kan løses ved hjælp af [Huggingface Transformers](http://huggingface.co).


Det første, vi skal gøre, er at vælge den model, vi vil bruge. Ud over nogle indbyggede modeller indeholder Huggingface et [online modelbibliotek](https://huggingface.co/models), hvor du kan finde mange flere forudtrænede modeller fra fællesskabet. Alle disse modeller kan indlæses og bruges blot ved at angive et modelnavn. Alle nødvendige binære filer til modellen vil automatisk blive downloadet.

På visse tidspunkter kan det være nødvendigt at indlæse dine egne modeller, i hvilket tilfælde du kan angive den mappe, der indeholder alle relevante filer, inklusive parametre til tokenizer, `config.json`-filen med modelparametre, binære vægte osv.

Ud fra modelnavnet kan vi oprette både modellen og tokenizer. Lad os starte med en tokenizer:


In [2]:
import transformers

# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
#bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

Objektet `tokenizer` indeholder funktionen `encode`, som kan bruges direkte til at kode tekst:


In [3]:
tokenizer.encode('Tensorflow is a great framework for NLP')

[101, 23435, 12314, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Vi kan også bruge tokenizer til at kode en sekvens på en måde, der er egnet til at sende til modellen, dvs. inklusive `token_ids`, `input_mask` felter osv. Vi kan også specificere, at vi ønsker Tensorflow-tensore ved at angive argumentet `return_tensors='tf'`:


In [4]:
tokenizer(['Hello, there'],return_tensors='tf')

{'input_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[ 101, 7592, 1010, 2045,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[1, 1, 1, 1, 1]], dtype=int32)>}

I vores tilfælde vil vi bruge den forudtrænede BERT-model kaldet `bert-base-uncased`. *Uncased* angiver, at modellen er case-insensitiv.

Når vi træner modellen, skal vi give en tokeniseret sekvens som input, og derfor vil vi designe en databehandlingspipeline. Da `tokenizer.encode` er en Python-funktion, vil vi bruge samme tilgang som i den sidste enhed ved at kalde den med `py_function`:


In [31]:
def process(x):
    return tokenizer.encode(x.numpy().decode('utf-8'),return_tensors='tf',padding='max_length',max_length=MAX_SEQ_LEN,truncation=True)[0]

def process_fn(x):
    s = x['title']+' '+x['description']
    e = tf.py_function(process,inp=[s],Tout=(tf.int32))
    e.set_shape(MAX_SEQ_LEN)
    return e,x['label']

Nu kan vi indlæse den faktiske model ved hjælp af `BertForSequenceClassification`-pakken. Dette sikrer, at vores model allerede har den nødvendige arkitektur til klassifikation, inklusive den endelige klassifikator. Du vil se en advarselsmeddelelse, der angiver, at vægtene for den endelige klassifikator ikke er initialiseret, og modellen vil kræve fortræning - det er helt i orden, fordi det er præcis, hvad vi er ved at gøre!


In [32]:
model = transformers.TFBertForSequenceClassification.from_pretrained(bert_model,num_labels=4,output_attentions=False)

In [33]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 109,485,316
Non-trainable params: 0
_________________________________________________________________


Som du kan se fra `summary()`, indeholder modellen næsten 110 millioner parametre! Formodentlig, hvis vi ønsker en simpel klassifikationsopgave på et relativt lille datasæt, ønsker vi ikke at træne BERT basislaget:


In [34]:
model.layers[0].trainable = False
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 3,076
Non-trainable params: 109,482,240
_________________________________________________________________


Nu er vi klar til at begynde træningen!

> **Note**: Træning af en fuldskala BERT-model kan være meget tidskrævende! Derfor vil vi kun træne den for de første 32 batches. Dette er blot for at vise, hvordan modeltræning sættes op. Hvis du er interesseret i at prøve fuldskala træning - fjern blot `steps_per_epoch` og `validation_steps` parametrene, og forbered dig på at vente!


In [30]:
model.compile('adam','sparse_categorical_crossentropy',['acc'])
tf.get_logger().setLevel('ERROR')
model.fit(ds_train.map(process_fn).batch(32),validation_data=ds_test.map(process_fn).batch(32),steps_per_epoch=32,validation_steps=2)

32/32 [==============================] - 142s 4s/step - loss: 1.3896 - acc: 0.2500 - val_loss: 1.3863 - val_acc: 0.2480


Hvis du øger antallet af iterationer og venter længe nok, og træner i flere epoker, kan du forvente, at BERT-klassifikation giver os den bedste nøjagtighed! Det skyldes, at BERT allerede forstår sprogets struktur ret godt, og vi behøver kun at finjustere den endelige klassifikator. Men fordi BERT er en stor model, tager hele træningsprocessen lang tid og kræver betydelig computerkraft! (GPU, og helst mere end én).

> **Note:** I vores eksempel har vi brugt en af de mindste forudtrænede BERT-modeller. Der findes større modeller, som sandsynligvis vil give bedre resultater.


## Vigtig pointe

I denne enhed har vi set på meget nyere modelarkitekturer baseret på **transformers**. Vi har anvendt dem til vores tekstklassificeringsopgave, men på samme måde kan BERT-modeller bruges til enhedsudtrækning, spørgsmål-svar og andre NLP-opgaver.

Transformer-modeller repræsenterer den nuværende state-of-the-art inden for NLP, og i de fleste tilfælde bør det være den første løsning, du eksperimenterer med, når du implementerer skræddersyede NLP-løsninger. Dog er det ekstremt vigtigt at forstå de grundlæggende principper bag rekurrente neurale netværk, som vi har diskuteret i dette modul, hvis du ønsker at bygge avancerede neurale modeller.



---

**Ansvarsfraskrivelse**:  
Dette dokument er blevet oversat ved hjælp af AI-oversættelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selvom vi bestræber os på nøjagtighed, skal du være opmærksom på, at automatiserede oversættelser kan indeholde fejl eller unøjagtigheder. Det originale dokument på dets oprindelige sprog bør betragtes som den autoritative kilde. For kritisk information anbefales professionel menneskelig oversættelse. Vi påtager os ikke ansvar for eventuelle misforståelser eller fejltolkninger, der opstår som følge af brugen af denne oversættelse.
